### Data Science Capstone Assignment week 3, part 1

## installing the necessary packages

In [8]:
pip install beautifulsoup4


     |████████████████████████████████| 112kB 6.7MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install lxml

     |████████████████████████████████| 5.8MB 26.7MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [14]:
from bs4 import BeautifulSoup
import bs4 as bs
import urllib.request

## Scraping data

In [15]:
sauce = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()
soup = bs.BeautifulSoup(sauce, 'lxml')

In [16]:
table = soup.table
print(table)

<table class="wikitable sortable">
<tbody><tr>
<th>Postal code
</th>
<th>Borough
</th>
<th>Neighborhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park / Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor / Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park / Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern / Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3B
</td>
<td>North York
</td>
<td>Don Mills
</td></tr>
<tr>
<td>M4B
</td>
<td>Ea

## Inserting data from the online table into a list that can be read into the df

In [88]:
data = []
for tr in table_rows:  
    td = tr.find_all('td') #find all table data
    row = [i.text for i in td] #add text to a list
    for i in row:  
        data.append(row) #append text of each row to the data list
print(data)

[['M1A\n', 'Not assigned\n', '\n'], ['M1A\n', 'Not assigned\n', '\n'], ['M1A\n', 'Not assigned\n', '\n'], ['M2A\n', 'Not assigned\n', '\n'], ['M2A\n', 'Not assigned\n', '\n'], ['M2A\n', 'Not assigned\n', '\n'], ['M3A\n', 'North York\n', 'Parkwoods\n'], ['M3A\n', 'North York\n', 'Parkwoods\n'], ['M3A\n', 'North York\n', 'Parkwoods\n'], ['M4A\n', 'North York\n', 'Victoria Village\n'], ['M4A\n', 'North York\n', 'Victoria Village\n'], ['M4A\n', 'North York\n', 'Victoria Village\n'], ['M5A\n', 'Downtown Toronto\n', 'Regent Park / Harbourfront\n'], ['M5A\n', 'Downtown Toronto\n', 'Regent Park / Harbourfront\n'], ['M5A\n', 'Downtown Toronto\n', 'Regent Park / Harbourfront\n'], ['M6A\n', 'North York\n', 'Lawrence Manor / Lawrence Heights\n'], ['M6A\n', 'North York\n', 'Lawrence Manor / Lawrence Heights\n'], ['M6A\n', 'North York\n', 'Lawrence Manor / Lawrence Heights\n'], ['M7A\n', 'Downtown Toronto\n', "Queen's Park / Ontario Provincial Government\n"], ['M7A\n', 'Downtown Toronto\n', "Queen's

In [89]:
import pandas as pd
from pandas import DataFrame

In [93]:
df = DataFrame(data, columns=['PostalCode','Borough','Neighbourhood']) #add columns
df.head() 

,PostalCode,Borough,Neighbourhood
0,M1A\n,Not assigned\n,\n
1,M1A\n,Not assigned\n,\n
2,M1A\n,Not assigned\n,\n
3,M2A\n,Not assigned\n,\n
4,M2A\n,Not assigned\n,\n


# Remove unassigned boroughs

In [97]:
indexNames = df[df['Borough'] == 'Not assigned\n'].index 
df.drop(indexNames, inplace=True) #remove not assigned boroughs
df.head()

,PostalCode,Borough,Neighbourhood
6,M3A\n,North York\n,Parkwoods\n
7,M3A\n,North York\n,Parkwoods\n
8,M3A\n,North York\n,Parkwoods\n
9,M4A\n,North York\n,Victoria Village\n
10,M4A\n,North York\n,Victoria Village\n


# Merge Neighbourhoods

In [104]:
temp_df=df.groupby('PostalCode')['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighbourhood':'Neighbourhood_joined'},inplace=True)

In [105]:

df_merge = pd.merge(df, temp_df, on='PostalCode')

In [107]:

df_merge.drop(['Neighbourhood'],axis=1,inplace=True)

In [108]:
df_merge.drop_duplicates(inplace=True)

In [109]:

df_merge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)

In [110]:
df_merge.head()

,PostalCode,Borough,Neighbourhood_joined
0,M3A\n,North York\n,"Parkwoods\n, Parkwoods\n, Parkwoods\n"
3,M4A\n,North York\n,"Victoria Village\n, Victoria Village\n, Victor..."
6,M5A\n,Downtown Toronto\n,"Regent Park / Harbourfront\n, Regent Park / Ha..."
9,M6A\n,North York\n,"Lawrence Manor / Lawrence Heights\n, Lawrence ..."
12,M7A\n,Downtown Toronto\n,Queen's Park / Ontario Provincial Government\n...


# Check shape of dataframe

In [111]:
df_merge.shape

(103, 3)